In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data Preprocessing by Sliding Window  
슬라이딩 윈도우 기법은 실시간 네트워크 패킷 데이터를 처리하는 기법에서, 데이터를 겹쳐 나눔으로써 전체 데이터가 증가하는 원리를 차용하여 데이터를 가공하는 기법이다.  
슬라이딩 윈도우 기법을 활용하면 많은 분석 데이터셋을 확보하고 학습데이터의 최근성을 가질 수 있다.  
ex) 인터넷 쇼핑몰 고객의 6개월 간의 구매내역, 방문 횟수 등의 데이터를 활용하여 6개월씩 데이터셋으로 만듬 -> A고객의 1월부터 6월 구매내역, 2월부터 7월 구매내역 ...

![sliding window](https://yxl-article.oss-cn-shenzhen.aliyuncs.com/images/sling-window.jpg)

In [1]:
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd

In [3]:
df_raw = pd.read_csv("../input/shopping-mall/shopping_mall.csv")

In [4]:
df_raw.head()

,cust_id,dt,ym,sale_amt
0,AFG4127094465,2021-12-09,202112,300000
1,AFG4127094465,2022-04-22,202204,45000
2,AFG4136725677,2022-04-07,202204,47800
3,AFG4140129314,2022-02-07,202202,39200
4,AFG4144923657,2022-05-11,202205,110000


In [16]:
# 각 시점 기준 컬럼 생성

# 날짜형으로 변환
df_raw['Date2']= pd.to_datetime(df_raw['dt']) 

# 시점변수 생성 
ym_li = [f"YM_M{i}" for i in range(6)]
ym_li

for i in range(len(ym_li)):
    col = ym_li[i]
    now = pd.to_datetime('2022-05-01') + relativedelta(months=-i)
# 현재시점 기준 미래(-) 과거(+)
    df_raw[col] = 'M' + (-np.floor((df_raw.Date2 - now)/(np.timedelta64(1, 'D')*31))).astype(int).astype(str) 
    
df_raw.head()

,cust_id,dt,ym,sale_amt,Date2,YM_M0,YM_M1,YM_M2,YM_M3,YM_M4,YM_M5
0,AFG4127094465,2021-12-09,202112,300000,2021-12-09,M5,M4,M3,M2,M1,M0
1,AFG4127094465,2022-04-22,202204,45000,2022-04-22,M1,M0,M-1,M-2,M-3,M-4
2,AFG4136725677,2022-04-07,202204,47800,2022-04-07,M1,M0,M-1,M-2,M-3,M-4
3,AFG4140129314,2022-02-07,202202,39200,2022-02-07,M3,M2,M1,M0,M-1,M-2
4,AFG4144923657,2022-05-11,202205,110000,2022-05-11,M0,M-1,M-2,M-3,M-4,M-5


*ValueError: Units 'M' and 'Y' are no longer supported*  
change `np.timedelta64(1, 'M'))` -> `np.timedelta64(1, 'D')*31)`

In [17]:
# 슬라이딩 윈도우 형태로 변환

m_col = ["M{}".format(i) for i in range(6)]   # M0~M5 목록 생성
df_li = []   # 임시테이블 저장할 목록

for n, ym in enumerate(ym_li):  # YM_M0 ~ YM_M5 반복
# STD_YM_M0 변수 기준 M0~M5 & 구매금액 0원 초과         
    tmp = df_raw[(df_raw[ym].isin(m_col)) & (df_raw['sale_amt'] > 0)] 
# YM_M0 기준 pivot
    tmp = tmp.pivot_table(index='cust_id', 
                          columns=ym, values='sale_amt', 
                          aggfunc='sum') 
    
    # M0~M12 중 누락된 컬럼 생성
    # 추후 테이블 union을 위해 pivot시 누락된 컬럼을 별도로 생성해줌
    missing_col = list(set(m_col) - set(tmp.columns)) 
    for col in missing_col :
        tmp[col] = 0
    
    # 컬럼이름 변경
    tmp.columns = [f'slae_amt_{c}' for c in tmp.columns] # 생략 가능
    
    tmp['MM_DIFF'] = ym
    tmp = tmp.fillna(0)

    df_li.append(tmp)

final_df = pd.concat(df_li).reset_index()
final_df.head()

,cust_id,slae_amt_M0,slae_amt_M1,slae_amt_M2,slae_amt_M3,slae_amt_M4,slae_amt_M5,MM_DIFF
0,AFG4127094465,0.0,45000.0,0.0,0.0,0.0,300000.0,YM_M0
1,AFG4136725677,0.0,47800.0,0.0,0.0,0.0,0.0,YM_M0
2,AFG4140129314,0.0,0.0,0.0,39200.0,0.0,0.0,YM_M0
3,AFG4144923657,110000.0,0.0,0.0,0.0,0.0,0.0,YM_M0
4,AFG4154711536,0.0,0.0,0.0,45000.0,0.0,0.0,YM_M0


In [18]:
# 특정 고객의 시점 별 형태 확인
df1 = final_df[(final_df['cust_id']=='AFG6825009314')]

df1

,cust_id,slae_amt_M0,slae_amt_M1,slae_amt_M2,slae_amt_M3,slae_amt_M4,slae_amt_M5,MM_DIFF
832,AFG6825009314,0.0,0.0,0.0,0.0,0.0,39200.0,YM_M0
1442,AFG6825009314,0.0,0.0,0.0,0.0,39200.0,0.0,YM_M1
1849,AFG6825009314,0.0,0.0,0.0,39200.0,0.0,0.0,YM_M2
2166,AFG6825009314,0.0,0.0,39200.0,0.0,0.0,0.0,YM_M3
2399,AFG6825009314,0.0,39200.0,0.0,0.0,0.0,0.0,YM_M4
2529,AFG6825009314,39200.0,0.0,0.0,0.0,0.0,0.0,YM_M5


`YM_M0` : 2022년 5월  
`sale_amt_M5` : 6달 전 구매금액

In [19]:
# 마지막 시점의 데이터 형태 확인
final_df.tail(10)

,cust_id,slae_amt_M0,slae_amt_M1,slae_amt_M2,slae_amt_M3,slae_amt_M4,slae_amt_M5,MM_DIFF
2523,AFG6544053758,78400.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2524,AFG6555046283,278000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2525,AFG6575203354,99000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2526,AFG6683774364,28000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2527,AFG6728543455,89000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2528,AFG6777275172,149000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2529,AFG6825009314,39200.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2530,AFG6838532849,99000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2531,AFG6846099516,129000.0,0.0,0.0,0.0,0.0,0.0,YM_M5
2532,AFG6860414162,139000.0,0.0,0.0,0.0,0.0,0.0,YM_M5


*마지막 시점 `MM_DIFF=YM_M5`의 데이터가 `sale_amt_M0`에만 값이 있는 이유는 기존 데이터가 6개월 동안의 구매 데이터만 존재하기 때문이다.*